In [1]:
# need python 3.12+ for typing
!python -V

Python 3.12.6


In [2]:
%load_ext autoreload
%autoreload 2

In [341]:
from src.sokoban_solver import (
    SokobanEdge,
    SokobanNode,
    Solver,
    BFSQueue,
    PriorityQueue,
)
from src.heuristics import (
    f_priority_length,
    f_priority_l1_naive,
    f_priority_l1_symmetric_naive,
    f_priority_l1_matching,
    f_priority_dijkstra,
    f_priority_dijkstra_stuck_basic,
    f_priority_dlsb,
)
from boards.utilities import load_json_to_dict

**Solver to do**
- Implement more heuristics
    - Chained heuristics: box pushing path length, via solving sub-problems
        - Include timeouts: box pushing distance, but if it times out we'll opt for board distance with a pruning condition
        - Considering stuck boxes to be equivalent to walls when making the sub problems
    - Splitting into sub puzzles?
- Create unit tests (runs solver on some test problems)
- requirements.txt
- What would it look like to work neural nets into the problem? As the sole heuristic? For blending heuristics? For a CNN? What's the right learning framework? Reinforcement?

**Generator to do**
- Learn the papers
    - https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=af6c8e280946f019c7f632c8281a35ddf1fdc9b0
    - https://ianparberry.com/techreports/LARC-2011-01.pdf
    - https://ieee-cog.org/2020/papers/paper_44.pdf
    - XSokoban test suite: https://sokoban-solver-statistics.sourceforge.io/statistics/XSokoban/XSokoban.html
    - Also do a bit more literature search
- Start implementing techniques

**Misc to do**
- Install htop: https://htop.dev/downloads.html

## Thinking

### Stuck

- Gets stuck already on this puzzle: https://linusakesson.net/games/autosokoban/?v=1&seed=1565730199&level=3
- A few clear issues here
    - It can't figure out the "easy" move of pushing a block down a hallway
    - One of the blocks just has to be pushed into the right location which should be trivial, but it's effectively pushing it randomly so good luck
- Easy heuristic idea
    - Priority queue by the minimum "empty board" solve distance: sum of L1 distances
    - Easy to have a test for this: a big empty board where you have to push the block 20 spaces to the goal
- Better heuristic
    - Instead of L1 distances, use "pushing this block as if it was the only block on the board" distance
    - The test would be pushing a block down a hall, around a U bend, and back down the "other side" hallway
    - This would naturally handle the case of a block which is stuck! Since a stuck block would have no way of pushing it to a goal.
    - Let's start with the other one first, then I need to think about "contract" stuff for how to compute the priorities with the new metric.
- OK actually I'm happy for now, I can get to trying more heuristics later

## Basic demos

In [4]:
filename = './boards/board_trivial.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node = SokobanNode(airs, boxes, goals, player)
print('Player component: ', node.player_component)
print('Avaiable pushes: ', node.edges)
print('Is solved?: ', node.solved)

print(node)

Player component:  {(0, 0)}
Avaiable pushes:  []
Is solved?:  True
█ █ █ █ █ █
█ █ █ █ █ █
█ █ ○ ■ █ █
█ █ █ █ █ █
█ █ █ █ █ █


In [5]:
filename = './boards/board_simple1.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node1 = SokobanNode(airs, boxes, goals, player)
print('Player component: ', node1.player_component)
print('Avaiable pushes: ', node1.edges)
print('Is solved?: ', node1.solved)

print(node1)

Player component:  {(0, 0)}
Avaiable pushes:  [SokobanEdge((1, 0), (0, 0), (1, 0), (2, 0))]
Is solved?:  False
█ █ █ █ █ █ █
█ █ █ █ █ █ █
█ █ ○ □ · █ █
█ █ █ █ █ █ █
█ █ █ █ █ █ █


In [6]:
filename = './boards/board_simple2.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node2 = SokobanNode(airs, boxes, goals, player)
print('Player component: ', node2.player_component)
print('Avaiable pushes: ', node2.edges)
print('Is solved?: ', node2.solved)

print(node2)

Player component:  {(0, 0)}
Avaiable pushes:  [SokobanEdge((0, 1), (0, 0), (0, 1), (0, 2)), SokobanEdge((1, 0), (0, 0), (1, 0), (2, 0))]
Is solved?:  False
█ █ █ █ █ █ █
█ █ █ █ █ █ █
█ █ · ·   █ █
█ █ □ □   █ █
█ █ ○ □ · █ █
█ █ █ █ █ █ █
█ █ █ █ █ █ █


In [7]:
filename = './boards/board_simple3.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node3 = SokobanNode(airs, boxes, goals, player)
print('Player component: ', node3.player_component)
print('Avaiable pushes: ', node3.edges)
print('Is solved?: ', node3.solved)

print(node3)

Player component:  {(4, 0), (3, 4), (4, 3), (3, 1), (0, 2), (1, 0), (1, 3), (4, 2), (3, 0), (3, 3), (0, 1), (2, 4), (1, 2), (0, 4), (2, 1), (4, 1), (4, 4), (0, 0), (1, 1), (0, 3), (2, 0), (1, 4)}
Avaiable pushes:  [SokobanEdge((1, 0), (1, 3), (2, 3), (3, 3)), SokobanEdge((-1, 0), (3, 3), (2, 3), (1, 3))]
Is solved?:  False
█ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █
█ █           █ █
█ █     ■ ·   █ █
█ █     □ █   █ █
█ █   ○       █ █
█ █           █ █
█ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █


In [8]:
node1 == node2, node1 == node3, node2 == node3

(False, False, False)

In [9]:
solver = Solver(BFSQueue(), node)
solver.solve()
print(solver.solution)

█ █ █ █ █ █
█ █ █ █ █ █
█ █ ○ ■ █ █
█ █ █ █ █ █
█ █ █ █ █ █

with no moves


In [11]:
solver = Solver(BFSQueue(), node1)
solver.solve()
print(solver.solution)

█ █ █ █ █ █ █
█ █ █ █ █ █ █
█ █ ○ □ · █ █
█ █ █ █ █ █ █
█ █ █ █ █ █ █

to

█ █ █ █ █ █ █
█ █ █ █ █ █ █
█ █   ○ ■ █ █
█ █ █ █ █ █ █
█ █ █ █ █ █ █

via

Move (1, 0) from (0, 0) to (1, 0)


In [12]:
solver = Solver(BFSQueue(), node2)
solver.solve()
print(solver.solution)

█ █ █ █ █ █ █
█ █ █ █ █ █ █
█ █ · ·   █ █
█ █ □ □   █ █
█ █ ○ □ · █ █
█ █ █ █ █ █ █
█ █ █ █ █ █ █

to

█ █ █ █ █ █ █
█ █ █ █ █ █ █
█ █ ■ ■   █ █
█ █   ○   █ █
█ █     ■ █ █
█ █ █ █ █ █ █
█ █ █ █ █ █ █

via

Move (0, 1) from (0, 0) to (0, 1)
Move (1, 0) from (0, 0) to (1, 0)
Move (0, 1) from (1, 0) to (1, 1)


In [13]:
solver = Solver(BFSQueue(), node3)
solver.solve()
print(solver.solution)

█ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █
█ █           █ █
█ █     ■ ·   █ █
█ █     □ █   █ █
█ █   ○       █ █
█ █           █ █
█ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █

to

█ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █
█ █           █ █
█ █     ■ ■   █ █
█ █     ○ █   █ █
█ █           █ █
█ █           █ █
█ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █

via

Move (1, 0) from (1, 3) to (2, 3)
Move (0, 1) from (2, 1) to (2, 2)


## Real puzzles

In [121]:
filename = './boards/board_open.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node = SokobanNode(airs, boxes, goals, player)
print(node)

█ █ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █                       █ █
█ █                   ·   █ █
█ █                       █ █
█ █                       █ █
█ █                       █ █
█ █           ○           █ █
█ █                       █ █
█ █                       █ █
█ █                       █ █
█ █   □                   █ █
█ █                       █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █ █


In [122]:
# fails
solver = Solver(BFSQueue(), node)
solver.solve(max_seconds=1)

301

In [123]:
# fails (it's just BFS!)
solver = Solver(PriorityQueue(f_priority_length), node)
solver.solve(max_seconds=1)

301

In [124]:
# succeeds!
solver = Solver(PriorityQueue(f_priority_l1_naive), node)
solver.solve(max_seconds=1)

200

In [125]:
print(solver.solution)

█ █ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █                       █ █
█ █                   ·   █ █
█ █                       █ █
█ █                       █ █
█ █                       █ █
█ █           ○           █ █
█ █                       █ █
█ █                       █ █
█ █                       █ █
█ █   □                   █ █
█ █                       █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █ █

to

█ █ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █                       █ █
█ █                   ■   █ █
█ █                   ○   █ █
█ █                       █ █
█ █                       █ █
█ █                       █ █
█ █                       █ █
█ █                       █ █
█ █                       █ █
█ █                       █ █
█ █                       █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █ █

via

Move (1, 0) from (0, 1) to (1, 1)
Move (1, 0) from (1, 1) to (2, 1)
Move (1, 0) from (2, 

In [229]:
filename = './boards/board_tricky_uturn.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node = SokobanNode(airs, boxes, goals, player)
print(node)

█ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █   ·                 █ █
█ █ █ █ █ █ █ █ █ █     █ █
█ █               █     █ █
█ █               █     █ █
█ █               █     █ █
█ █               █     █ █
█ █ ○ □                 █ █
█ █                     █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █


In [273]:
# usual heuristic takes lots of iters (but it does work quickly)
solver = Solver(PriorityQueue(f_priority_l1_matching), node)
solver.solve(max_iters=30)

302

In [275]:
# dijkstra heuristic gets it quickly!
solver = Solver(PriorityQueue(f_priority_dijkstra), node)
solver.solve(max_iters=30)

200

https://linusakesson.net/games/autosokoban/?v=1&seed=1565730199&level=1

In [126]:
filename = './boards/board_v=1&seed=1565730199&level=1.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node = SokobanNode(airs, boxes, goals, player)
print(node)

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █       █ █ █ █ █ █ █ █
█ █   █ · █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █       █           █ █
█ █ █     █   █ █ █   █ █
█ █   □ ■       █ █   █ █
█ █ █   ○ █           █ █
█ █ █ █ █ █       █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █


In [127]:
solver = Solver(PriorityQueue(f_priority_l1_naive), node)
solver.solve()
print(solver.solution)

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █       █ █ █ █ █ █ █ █
█ █   █ · █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █       █           █ █
█ █ █     █   █ █ █   █ █
█ █   □ ■       █ █   █ █
█ █ █   ○ █           █ █
█ █ █ █ █ █       █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █

to

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █       █ █ █ █ █ █ █ █
█ █   █ ■ █ █ █ █ █ █ █ █
█ █   █ ○ █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █       █           █ █
█ █ █     █   █ █ █   █ █
█ █     ■       █ █   █ █
█ █ █     █           █ █
█ █ █ █ █ █       █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █

via

Move (0, 1) from (1, 1) to (1, 2)
Move (0, 1) from (1, 2) to (1, 3)
Move (1, 0) from (0, 4) to (1, 4)
Move (0, 1) from (2, 3) to (2, 4)
Move (0, 1) from (2, 4) to (2, 5)
Move (0, 1) from (2, 5) to (2, 6)


https://linusakesson.net/games/autosokoban/?v=1&seed=1565730199&level=2

In [128]:
filename = './boards/board_v=1&seed=1565730199&level=2.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node = SokobanNode(airs, boxes, goals, player)
print(node)

█ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ ○ █ █ █ █ █ █ █ █ █ █
█ █ █ □ █ █ █ █ █ █ █ █ █ █
█ █ █   █ █ █     █ █ █ █ █
█ █ ·   □   █     █ █ █ █ █
█ █     █   █         █ █ █
█ █   · █     ·   □     █ █
█ █   █ █   █     █   █ █ █
█ █                   █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █


In [129]:
solver = Solver(PriorityQueue(f_priority_l1_naive), node)
solver.solve()
print(solver.solution)

█ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ ○ █ █ █ █ █ █ █ █ █ █
█ █ █ □ █ █ █ █ █ █ █ █ █ █
█ █ █   █ █ █     █ █ █ █ █
█ █ ·   □   █     █ █ █ █ █
█ █     █   █         █ █ █
█ █   · █     ·   □     █ █
█ █   █ █   █     █   █ █ █
█ █                   █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █

to

█ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █   █ █ █ █ █ █ █ █ █ █
█ █ █   █ █ █ █ █ █ █ █ █ █
█ █ █   █ █ █     █ █ █ █ █
█ █ ■       █     █ █ █ █ █
█ █     █   █         █ █ █
█ █   ■ █     ■ ○       █ █
█ █   █ █   █     █   █ █ █
█ █                   █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █ █

via

Move (0, -1) from (2, 8) to (2, 7)
Move (0, -1) from (2, 7) to (2, 6)
Move (0, -1) from (2, 6) to (2, 5)
Move (-1, 0) from (9, 3) to (8, 3)
Move (0, -1) from (2, 5) to (2, 4)
Move (-1, 0) from (4, 5) to (3, 5)
Move (-1, 0) from (3, 5) to (2, 5)
Move (-1, 0) from (8, 3) to (7, 3)


https://linusakesson.net/games/autosokoban/?v=1&seed=1565730199&level=3

In [130]:
filename = './boards/board_v=1&seed=1565730199&level=3.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node = SokobanNode(airs, boxes, goals, player)
print(node)

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █         █ █
█ █ █ █ █ █ █   █ □   █ █
█ █ ·         ·       █ █
█ █ █ █ █ █ █   □     █ █
█ █ █ █ █ █ █     █   █ █
█ █ █ █ █ █ █         █ █
█ █ █ █ █ █ █ █   █   █ █
█ █ █ █ █ █ █ █ ○   ■ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █


In [131]:
# fails with naive l1 priority
solver = Solver(PriorityQueue(f_priority_l1_naive), node)
solver.solve(max_seconds=1)

301

In [132]:
# succeeds with symmetric naive l1 priority!
solver = Solver(PriorityQueue(f_priority_l1_symmetric_naive), node)
solver.solve(max_seconds=1)

200

In [133]:
# succeeds with l1 matching priority!
solver = Solver(PriorityQueue(f_priority_l1_matching), node)
solver.solve(max_seconds=1)

200

In [134]:
print(solver.solution)

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █         █ █
█ █ █ █ █ █ █   █ □   █ █
█ █ ·         ·       █ █
█ █ █ █ █ █ █   □     █ █
█ █ █ █ █ █ █     █   █ █
█ █ █ █ █ █ █         █ █
█ █ █ █ █ █ █ █   █   █ █
█ █ █ █ █ █ █ █ ○   ■ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █

to

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █         █ █
█ █ █ █ █ █ █   █     █ █
█ █ ■         ■       █ █
█ █ █ █ █ █ █ ○       █ █
█ █ █ █ █ █ █     █   █ █
█ █ █ █ █ █ █         █ █
█ █ █ █ █ █ █ █   █   █ █
█ █ █ █ █ █ █ █     ■ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █

via

Move (0, -1) from (7, 7) to (7, 6)
Move (-1, 0) from (8, 5) to (7, 5)
Move (-1, 0) from (7, 5) to (6, 5)
Move (-1, 0) from (6, 5) to (5, 5)
Move (-1, 0) from (5, 5) to (4, 5)
Move (-1, 0) from (4, 5) to (3, 5)
Move (-1, 0) from (3, 5) to (2, 5)
Move (-1, 0) from (7, 4) to (6, 4)
Move (-1, 0) from (2, 5) to (1, 5)
Move (0, 1) from (5, 3) to (5, 4)


https://linusakesson.net/games/autosokoban/?v=1&seed=1565730199&level=4

In [142]:
filename = './boards/board_v=1&seed=1565730199&level=4.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node = SokobanNode(airs, boxes, goals, player)
print(node)

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █       █ █ █ █ █ █
█ █   □ █   █     █ █ █
█ █           · ○   █ █
█ █   □         ·   █ █
█ █   ■ █         █ █ █
█ █ □ █ █ █   █ █ █ █ █
█ █   █ █ █   █ █ █ █ █
█ █ · █ █ █   █ █ █ █ █
█ █   █ █ █   █ █ █ █ █
█ █   █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █


In [145]:
# fails!
solver = Solver(PriorityQueue(f_priority_l1_symmetric_naive), node)
solver.solve(max_seconds=1)

301

In [146]:
# works!
solver = Solver(PriorityQueue(f_priority_l1_matching), node)
solver.solve(max_seconds=1)

200

In [147]:
print(solver.solution)

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █       █ █ █ █ █ █
█ █   □ █   █     █ █ █
█ █           · ○   █ █
█ █   □         ·   █ █
█ █   ■ █         █ █ █
█ █ □ █ █ █   █ █ █ █ █
█ █   █ █ █   █ █ █ █ █
█ █ · █ █ █   █ █ █ █ █
█ █   █ █ █   █ █ █ █ █
█ █   █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █

to

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █       █ █ █ █ █ █
█ █     █   █     █ █ █
█ █         ○ ■     █ █
█ █             ■   █ █
█ █   ■ █         █ █ █
█ █   █ █ █   █ █ █ █ █
█ █   █ █ █   █ █ █ █ █
█ █ ■ █ █ █   █ █ █ █ █
█ █   █ █ █   █ █ █ █ █
█ █   █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █

via

Move (1, 0) from (0, 6) to (1, 6)
Move (0, -1) from (0, 5) to (0, 4)
Move (0, -1) from (0, 4) to (0, 3)
Move (0, -1) from (1, 9) to (1, 8)
Move (1, 0) from (0, 7) to (1, 7)
Move (1, 0) from (1, 6) to (2, 6)
Move (1, 0) from (1, 7) to (2, 7)
Move (1, 0) from (2, 7) to (3, 7)
Move (1, 0) from (2, 6) to (3, 6)
Move (1,

https://linusakesson.net/games/autosokoban/?v=1&seed=1565730199&level=5

In [313]:
filename = './boards/board_v=1&seed=1565730199&level=5.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node = SokobanNode(airs, boxes, goals, player)
print(node)

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █         ● █ █ █
█ █ █ █ █ · █ █ █   █ █ █
█ █ █ █ █     █ █ □ █ █ █
█ █               □   █ █
█ █   █ █   █   █ █   █ █
█ █   █ █   █         █ █
█ █   █ █     ■ □ █ █ █ █
█ █           ·   █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █


In [315]:
# fails :(
solver = Solver(PriorityQueue(f_priority_dijkstra), node)
solver.solve(max_seconds=1)

301

In [318]:
# succeeds!
solver = Solver(PriorityQueue(f_priority_dijkstra_stuck_basic), node)
solver.solve(max_seconds=1)

200

https://linusakesson.net/games/autosokoban/?v=1&seed=1565730199&level=11

The solution for this one is truly devious. So I'm not surprised the solver is unable to find it. But somehow Linus Akesson's solver made in like 2008 can get it, so clearly I'm doing something wrong.

In [319]:
filename = './boards/board_v=1&seed=1565730199&level=11.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node = SokobanNode(airs, boxes, goals, player)
print(node)

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █   █ █ █ █ █ █
█ █ █   ·         █ █ █
█ █ █   █   □ · □   █ █
█ █ · □ □   ■ ·     █ █
█ █     □   █ █ █ █ █ █
█ █   █ █   █ █ █ █ █ █
█ █ · ○     █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █


In [327]:
# fails
solver = Solver(PriorityQueue(f_priority_l1_naive), node)
solver.solve(max_seconds=1)

301

In [328]:
# pretty insane searching
solver.n_iters_overall, solver.n_revisited_overall, len(solver.queue)

(6427, 15347, 4916)

In [329]:
# dumb attempt
print(solver.queue[0])

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █   █ █ █ █ █ █
█ █ █   ·         █ █ █
█ █ █   █   □ · □   █ █
█ █ · □ □   ■ ·     █ █
█ █     □   █ █ █ █ █ █
█ █   █ █   █ █ █ █ █ █
█ █ · ○     █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █

to

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █   █ █ █ █ █ █
█ █ █   ·   ○ □   █ █ █
█ █ █   █     ■     █ █
█ █ ■   □ □ ■ ·     █ █
█ █         █ █ █ █ █ █
█ █   █ █   █ █ █ █ █ █
█ █ ·       █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █

via

Move (1, 0) from (3, 4) to (4, 4)
Move (-1, 0) from (3, 2) to (2, 2)
Move (-1, 0) from (2, 2) to (1, 2)
Move (0, 1) from (0, 1) to (0, 2)
Move (0, -1) from (1, 4) to (1, 3)
Move (1, 0) from (1, 3) to (2, 3)
Move (0, 1) from (3, 2) to (3, 3)
Move (0, 1) from (3, 3) to (3, 4)
Move (-1, 0) from (4, 5) to (3, 5)
Move (0, -1) from (6, 5) to (6, 4)
Move (-1, 0) from (6, 4) to (5, 4)
Move (-1, 0) from (5, 3) to (4, 3)
Move (1, 0) from (3, 4) to (4, 4)
Move (-1, 0)

In [330]:
# fails
solver = Solver(PriorityQueue(f_priority_dijkstra), node)
solver.solve(max_seconds=1)

301

In [331]:
# much more reasonable searching
solver.n_iters_overall, solver.n_revisited_overall, len(solver.queue)

(488, 947, 334)

In [332]:
# reasonable attempt!
print(solver.queue[0])

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █   █ █ █ █ █ █
█ █ █   ·         █ █ █
█ █ █   █   □ · □   █ █
█ █ · □ □   ■ ·     █ █
█ █     □   █ █ █ █ █ █
█ █   █ █   █ █ █ █ █ █
█ █ · ○     █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █

to

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █   █ █ █ █ █ █
█ █ █ □ ·         █ █ █
█ █ █   █     ■     █ █
█ █ ■     □ ■ ●     █ █
█ █         █ █ █ █ █ █
█ █   █ █   █ █ █ █ █ █
█ █ ■       █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █

via

Move (1, 0) from (3, 4) to (4, 4)
Move (-1, 0) from (3, 2) to (2, 2)
Move (-1, 0) from (2, 2) to (1, 2)
Move (0, 1) from (1, 2) to (1, 3)
Move (0, -1) from (0, 3) to (0, 2)
Move (-1, 0) from (3, 3) to (2, 3)
Move (0, -1) from (0, 2) to (0, 1)
Move (-1, 0) from (2, 3) to (1, 3)
Move (0, -1) from (6, 5) to (6, 4)
Move (0, 1) from (1, 3) to (1, 4)
Move (-1, 0) from (6, 4) to (5, 4)
Move (-1, 0) from (5, 3) to (4, 3)
Move (1, 0) from (3, 4) to (4, 4)
Move (-1, 

In [335]:
# fails
solver = Solver(PriorityQueue(f_priority_dijkstra_stuck_basic), node)
solver.solve(max_seconds=1)

301

In [336]:
# again a reasonable queue
solver.n_iters_overall, solver.n_revisited_overall, len(solver.queue)

(427, 926, 413)

In [337]:
# reasonable attempt!
print(solver.queue[0])

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █   █ █ █ █ █ █
█ █ █   ·         █ █ █
█ █ █   █   □ · □   █ █
█ █ · □ □   ■ ·     █ █
█ █     □   █ █ █ █ █ █
█ █   █ █   █ █ █ █ █ █
█ █ · ○     █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █

to

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █   █ █ █ █ █ █
█ █ █   · □       █ █ █
█ █ █ □ █   □ ●     █ █
█ █ ·       ■ ■     █ █
█ █         █ █ █ █ █ █
█ █ □ █ █   █ █ █ █ █ █
█ █ ·       █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █

via

Move (1, 0) from (3, 4) to (4, 4)
Move (-1, 0) from (3, 2) to (2, 2)
Move (-1, 0) from (2, 2) to (1, 2)
Move (0, 1) from (1, 2) to (1, 3)
Move (0, -1) from (0, 3) to (0, 2)
Move (-1, 0) from (3, 3) to (2, 3)
Move (0, -1) from (6, 5) to (6, 4)
Move (-1, 0) from (7, 3) to (6, 3)
Move (1, 0) from (0, 3) to (1, 3)
Move (1, 0) from (1, 3) to (2, 3)
Move (0, 1) from (3, 2) to (3, 3)
Move (0, 1) from (3, 3) to (3, 4)
Move (-1, 0) from (6, 4) to (5, 4)


In [352]:
# fails
solver = Solver(PriorityQueue(f_priority_dlsb), node)
solver.solve(max_seconds=1)

301

In [353]:
# reasonable enough attempt
print(solver.queue[0])

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █   █ █ █ █ █ █
█ █ █   ·         █ █ █
█ █ █   █   □ · □   █ █
█ █ · □ □   ■ ·     █ █
█ █     □   █ █ █ █ █ █
█ █   █ █   █ █ █ █ █ █
█ █ · ○     █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █

to

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █   █ █ █ █ █ █
█ █ █   ■         █ █ █
█ █ █   █   □ ●     █ █
█ █ ·       ■ ■     █ █
█ █         █ █ █ █ █ █
█ █ □ █ █   █ █ █ █ █ █
█ █ ■       █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █

via

Move (1, 0) from (3, 4) to (4, 4)
Move (-1, 0) from (3, 2) to (2, 2)
Move (-1, 0) from (2, 2) to (1, 2)
Move (0, 1) from (1, 2) to (1, 3)
Move (0, -1) from (0, 3) to (0, 2)
Move (-1, 0) from (3, 3) to (2, 3)
Move (0, -1) from (0, 2) to (0, 1)
Move (0, -1) from (6, 5) to (6, 4)
Move (-1, 0) from (7, 3) to (6, 3)
Move (1, 0) from (0, 3) to (1, 3)
Move (0, -1) from (1, 5) to (1, 4)
Move (0, -1) from (1, 4) to (1, 3)
Move (1, 0) from (1, 3) to (2, 3)
Move (-1, 

## Bottom

In [ ]:
solver.step()
print(len(solver.queue))
print()
print(solver.queue.get_str_end_nodes())

In [289]:
from src.heuristics import check_for_stuck_boxes

In [303]:
node = SokobanNode({(0, 0), (0, 1), (1, 0), (1, 1), (-1, 0), (0, -1)}, {(0, 0), (0, 1)}, {(1, 1), (0, 1)}, (0, 0))
print(node)

█ █ █ █ █ █ █
█ █ █ █ █ █ █
█ █ █ ■ · █ █
█ █   ○   █ █
█ █ █   █ █ █
█ █ █ █ █ █ █
█ █ █ █ █ █ █


In [304]:
check_for_stuck_boxes(node)

False